In [1]:
import tensorflow
import cv2
import numpy as np
from tensorflow.keras import layers, models
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [7]:
import os
import cv2
import numpy as np  # Import NumPy

def load_process(path, target_size=(64, 64)):
    images = []  # List to store loaded images

    # Loop through all files in the directory
    for filename in os.listdir(path):
        # Check for valid image file types
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Create the full path to the image file
            img_path = os.path.join(path, filename)

            # Load the image using OpenCV
            img = cv2.imread(img_path)

            # Check if the image was loaded successfully
            if img is not None:
                # Resize image to target size
                img = cv2.resize(img, target_size)
                # Normalize pixel values to [0, 1]
                img = img / 255.0
                images.append(img)  # Store the loaded image
            else:
                print(f"Failed to load {filename}")

    return np.array(images)


path = "/content/drive/MyDrive/data/chest_xray/test/PNEUMONIA"
loaded_images = load_process(path)

print(f"Loaded images shape: {loaded_images.shape}")


if loaded_images.size > 0:



    try:
        embedding = emb_model.predict(loaded_images)
        print(f"Embeddings shape: {embedding.shape}")
    except ValueError as e:
        print(f"Error during prediction: {e}")
else:
    print("No images were loaded.")


Loaded images shape: (390, 64, 64, 3)
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
Embeddings shape: (390, 128)


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models


def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))  # Embedding layer
    return model


In [6]:

emb_model = create_model()


embedding = emb_model.predict(loaded_images)

print(f"Embeddings shape: {embedding.shape}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step
Embeddings shape: (390, 128)


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU

def build_generator(embedding):
    model = Sequential()

    # Input layer
    model.add(Dense(128 * 8 * 8, input_dim=embedding_dim))
    model.add(Reshape((8, 8, 128)))  # Reshape to 3D tensor (height, width, channels)

    # Upsampling
    model.add(Conv2DTranspose(64, (4, 4), strides=2, padding='same'))  # Upsample to 16x16
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(32, (4, 4), strides=2, padding='same'))  # Upsample to 32x32
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh'))  # Output image: 64x64x3

    return model


In [9]:
def train_gan(epochs, batch_size, loaded_images, embeddings):
    for epoch in range(epochs):

        idx = np.random.randint(0, loaded_images.shape[0], batch_size)
        real_images = loaded_images[idx]


        embedding_idx = np.random.randint(0, embeddings.shape[0], batch_size)
        input_embeddings = embeddings[embedding_idx]


        fake_images = generator.predict(input_embeddings)


        d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))


        noise = np.random.normal(0, 1, (batch_size, embedding_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        if epoch % 100 == 0:
            print(f"Epoch: {epoch}, D Loss: {0.5 * (d_loss_real + d_loss_fake)}, G Loss: {g_loss}")


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_discriminator(input_shape=(64, 64, 3)):
    model = models.Sequential()

    # Convolutional Layer 1
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha=0.2))  # Use Leaky ReLU activation
    model.add(layers.Dropout(0.3))  # Dropout for regularization

    # Convolutional Layer 2
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))  # Downsample

    # Convolutional Layer 3
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten and Fully Connected Layer
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    # Output Layer
    model.add(layers.Dense(1, activation='sigmoid'))  # Output is probability of real/fake

    return model

# Example usage:
discriminator = build_discriminator()
discriminator.summary()  # Print model summary


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │      33,554,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,926,273 (129.42 MB)

 Trainable params: 33,926,273 (129.42 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
embedding_dim=128

In [12]:
generator = build_generator(embedding)
discriminator = build_discriminator()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


gan_input = layers.Input(shape=(embedding_dim,))  # Input for GAN
fake_images = generator(gan_input)  # Generating images from the input
discriminator.trainable = False  # Freezing discriminator during generator training
gan_output = discriminator(fake_images)  # Passing fake images to the discriminator
gan = models.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')


In [17]:
import tensorflow as tf

def generate_gradcam(discriminator, image):
    with tf.GradientTape() as tape:
        conv_outputs, predictions = discriminator(image)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs), axis=-1)


    cam = np.maximum(cam, 0) / np.max(cam)
    cam = cv2.resize(cam.numpy(), (64, 64))
    return cam
